In [1]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [3]:
#bring in CSV 
census_df = pd.read_csv("census_df_isoto-Copy1.csv")
census_df.head()

,Unnamed: 0,Name,Year,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Count Hisp/Latino,Counb Black/AfAm,Count AmIndian,Count Asian,Count White,Count Other Race,Zipcode
0,0,ZCTA5 77002,2012,57165.0,11473.0,31.8,32514.0,813.0,3175.0,4002.0,6.0,232.0,4018.0,0.0,77002
1,1,ZCTA5 77003,2012,37664.0,10215.0,32.9,27248.0,3868.0,5351.0,2063.0,0.0,443.0,2318.0,26.0,77003
2,2,ZCTA5 77004,2012,39804.0,29853.0,31.6,29787.0,6957.0,3358.0,16520.0,10.0,2149.0,7198.0,120.0,77004
3,3,ZCTA5 77005,2012,153041.0,25740.0,38.5,88579.0,635.0,2076.0,746.0,25.0,2484.0,19856.0,60.0,77005
4,4,ZCTA5 77006,2012,63360.0,19856.0,36.9,60795.0,2647.0,3816.0,695.0,38.0,1127.0,13874.0,66.0,77006


In [4]:
#see if there are null values in data
census_df.isnull().values.any()

False

In [5]:
#do an ANOVA test! look at zip codes, and the household income changes per race

#want to look at differences etween group means, the groups being years and changes of race and income in those zips
#split up data into years
yr2012 = census_df[census_df["Year"] ==2012]
# yr2013 = census_df[census_df["Year"]==2013]
# yr2014 = census_df[census_df["Year"] ==2014]
# yr2015 = census_df[census_df["Year"]==2015]
# yr2016 = census_df[census_df["Year"]==2016]
# yr2017 = census_df[census_df["Year"]==2017]

df_test = yr2012.groupby(["Zipcode"])
hisp_lat_count = df_test["Count Hisp/Latino"].sum()
af_am_count = df_test["Counb Black/AfAm"].sum()


# group2 = census_pd[census_pd["Zipcode"]== "77003"]["Counb Black/AfAm"]
# group3 = census_pd[census_pd["Zipcode"]== "77030"]["Counb Black/AfAm"]

#df=yr2012.pivot_table(df,index='Count Hisp/Latino',columns='platform',values='count')

yr2012.head()

,Unnamed: 0,Name,Year,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Count Hisp/Latino,Counb Black/AfAm,Count AmIndian,Count Asian,Count White,Count Other Race,Zipcode
0,0,ZCTA5 77002,2012,57165.0,11473.0,31.8,32514.0,813.0,3175.0,4002.0,6.0,232.0,4018.0,0.0,77002
1,1,ZCTA5 77003,2012,37664.0,10215.0,32.9,27248.0,3868.0,5351.0,2063.0,0.0,443.0,2318.0,26.0,77003
2,2,ZCTA5 77004,2012,39804.0,29853.0,31.6,29787.0,6957.0,3358.0,16520.0,10.0,2149.0,7198.0,120.0,77004
3,3,ZCTA5 77005,2012,153041.0,25740.0,38.5,88579.0,635.0,2076.0,746.0,25.0,2484.0,19856.0,60.0,77005
4,4,ZCTA5 77006,2012,63360.0,19856.0,36.9,60795.0,2647.0,3816.0,695.0,38.0,1127.0,13874.0,66.0,77006


In [6]:
# drop unnamed and name columns 
data = census_df[['Zipcode', 'Year',
            'Count Hisp/Latino',
            'Counb Black/AfAm', 
             'Count AmIndian', 
             'Count Asian', 
             'Count White',
            'Count Other Race','Household Income', 'Population', 'Median Age',
            'Per Capita Income', 'Poverty Count', ]]

# MELT the data
melted_df = data.melt(id_vars=['Zipcode','Year', 'Household Income', 
                               'Population','Median Age',
                               'Per Capita Income', 'Poverty Count'], var_name='Race', value_name='Race Count')

# sort it back by zipcode and Year 
final_df = melted_df.sort_values(['Zipcode', 'Year'], ascending=[True, True])
# reset index
final_df = final_df.reset_index(drop=True)

# print first five rows
final_df.head()


,Zipcode,Year,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Race,Race Count
0,77002,2012,57165.0,11473.0,31.8,32514.0,813.0,Count Hisp/Latino,3175.0
1,77002,2012,57165.0,11473.0,31.8,32514.0,813.0,Counb Black/AfAm,4002.0
2,77002,2012,57165.0,11473.0,31.8,32514.0,813.0,Count AmIndian,6.0
3,77002,2012,57165.0,11473.0,31.8,32514.0,813.0,Count Asian,232.0
4,77002,2012,57165.0,11473.0,31.8,32514.0,813.0,Count White,4018.0


In [54]:
#the three zip codes we want to look at
zip_003 = final_df[final_df["Zipcode"] ==77003]
zip_007 = final_df[final_df["Zipcode"]==77007]
zip_008 = final_df[final_df["Zipcode"]==77008]
zip_026 = final_df[final_df["Zipcode"]==77026]

In [8]:
#zip per year
zip003_yr2012 = zip_003[zip_003["Year"]== 2012]
zip003_yr2013 = zip_003[zip_003["Year"]== 2013]
zip003_yr2014 = zip_003[zip_003["Year"]== 2014]
zip003_yr2015 = zip_003[zip_003["Year"]== 2015]
zip003_yr2016 = zip_003[zip_003["Year"]== 2016]
zip003_yr2017 = zip_003[zip_003["Year"]== 2017]
# yr2013 = census_df[census_df["Year"]==2013]
# yr2014 = census_df[census_df["Year"] ==2014]
# yr2015 = census_df[census_df["Year"]==2015]
# yr2016 = census_df[census_df["Year"]==2016]
# yr2017 = census_df[census_df["Year"]==2017]
zip_003


,Zipcode,Year,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Race,Race Count
36,77003,2012,37664.0,10215.0,32.9,27248.0,3868.0,Count Hisp/Latino,5351.0
37,77003,2012,37664.0,10215.0,32.9,27248.0,3868.0,Counb Black/AfAm,2063.0
38,77003,2012,37664.0,10215.0,32.9,27248.0,3868.0,Count AmIndian,0.0
39,77003,2012,37664.0,10215.0,32.9,27248.0,3868.0,Count Asian,443.0
40,77003,2012,37664.0,10215.0,32.9,27248.0,3868.0,Count White,2318.0
41,77003,2012,37664.0,10215.0,32.9,27248.0,3868.0,Count Other Race,26.0
42,77003,2013,36412.0,10617.0,30.8,27263.0,4130.0,Count Hisp/Latino,5615.0
43,77003,2013,36412.0,10617.0,30.8,27263.0,4130.0,Counb Black/AfAm,1885.0
44,77003,2013,36412.0,10617.0,30.8,27263.0,4130.0,Count AmIndian,0.0
45,77003,2013,36412.0,10617.0,30.8,27263.0,4130.0,Count Asian,587.0


In [9]:
#analysis of group differences of a race within zip codes per year
#EDO HISP LATINO PER YEAR DOESN'T WORK, ASK SATURDAY
# test_group1 = zip003_yr2012[zip003_yr2012["Race"] == "Count Hisp/Latino"]["Household Income"]
# test_group2 = zip003_yr2013[zip003_yr2013["Race"] == "Count Hisp/Latino"]["Household Income"]
# test_group3 = zip003_yr2014[zip003_yr2014["Race"] == "Count Hisp/Latino"]["Household Income"]
# test_group4 = zip003_yr2015[zip003_yr2015["Race"] == "Count Hisp/Latino"]["Household Income"]
# test_group5 = zip003_yr2016[zip003_yr2016["Race"] == "Count Hisp/Latino"]["Household Income"]
# test_group6 = zip003_yr2017[zip003_yr2017["Race"] == "Count Hisp/Latino"]["Household Income"]


In [10]:
# stats.f_oneway(test_group1, test_group2, test_group3, test_group4, test_group5, test_group6)

In [11]:
#Analysis of household income for race per zip 77003
## won't work, because we don't have income for each race per zip

# group1 = zip_003[zip_003["Race"] == "Count Hisp/Latino"]["Household Income"]
# group2 = zip_003[zip_003["Race"] == "Counb Black/AfAm"]["Household Income"]
# group3 = zip_003[zip_003["Race"] == "Count White"]["Household Income"]
# group4 = zip_003[zip_003["Race"] == "Count Asian"]["Household Income"]
# group5 = zip_003[zip_003["Race"] == "Count AmIndian"]["Household Income"]
# group6 = zip_003[zip_003["Race"] == "Count Other Race"]["Household Income"]

# #analysis of race per zip 77007
# group1_007 = zip_007[zip_007["Race"] == "Count Hisp/Latino"]["Household Income"]
# group2_007 = zip_007[zip_007["Race"] == "Counb Black/AfAm"]["Household Income"]
# group3_007 = zip_007[zip_007["Race"] == "Count White"]["Household Income"]
# group4_007 = zip_007[zip_007["Race"] == "Count Asian"]["Household Income"]
# group5_007 = zip_007[zip_007["Race"] == "Count AmIndian"]["Household Income"]
# group6_007 = zip_007[zip_007["Race"] == "Count Other Race"]["Household Income"]

In [12]:
#analysis of group differences of a race within a zip 77003
#why does p value equal nan?
# ANOVA_003 = stats.f_oneway(group1, group2, group3, group4, group5, group6)
# ANOVA_003

In [13]:
# ANOVA_007 = stats.f_oneway(group1_007, group2_007, group3_007, group4_007, group5_007, group6_007)
# ANOVA_007

In [14]:
#analysis of vairance zip and population

pop_003 = census_df[census_df["Zipcode"]==77003]["Population"]
pop_008 = census_df[census_df["Zipcode"]==77008]["Population"]
pop_007 = census_df[census_df["Zipcode"]==77007]["Population"]
pop_026 = census_df[census_df["Zipcode"]==77026]["Population"]


In [15]:
ANOVA_pop = stats.f_oneway(pop_003, pop_008, pop_007, pop_026)
# there is a statistically significant difference between and within population counts and zip codes from 2012 - 2017

#living in an inner loop zip code matters when it comes to the populaiton count
ANOVA_pop

F_onewayResult(statistic=278.56408336054585, pvalue=1.7810577568661499e-16)

In [16]:
#analysis of variance zip and age
age_003 = census_df[census_df["Zipcode"]==77003]["Median Age"]
age_008 = census_df[census_df["Zipcode"]==77008]["Median Age"]
age_007 = census_df[census_df["Zipcode"]==77007]["Median Age"]
age_026 = census_df[census_df["Zipcode"]==77026]["Median Age"]


In [17]:
ANOVA_age = stats.f_oneway(age_003, age_008, age_007, age_026)
# there is a statistically significant difference between and within median age and zip codes from 2012 - 2017

#living in one of the inner loop zip codes matters when it comes to determining median age
ANOVA_age

F_onewayResult(statistic=10.711387129500133, pvalue=0.0002060830183355059)

In [65]:
#analysis of variance zip and household income
income_003 = census_df[census_df["Zipcode"]==77003]["Household Income"]
income_008 = census_df[census_df["Zipcode"]==77008]["Household Income"]
income_007 = census_df[census_df["Zipcode"]==77007]["Household Income"]
income_026 = census_df[census_df["Zipcode"]==77026]["Household Income"]



5       90860.0
24      98725.0
43     106044.0
62     107580.0
81     113361.0
100    121458.0
Name: Household Income, dtype: float64

In [64]:
ANOVA_income = stats.f_oneway(income_003, income_008, income_007, income_026)

# there is a statistically significant difference between and within household income (median income) counts and zip codes from 2012 - 2017

#this one below is the best!
#living in a zip code matters when it comes to household income (median income)
ANOVA_income

F_onewayResult(statistic=99.24201522961962, pvalue=3.5090707198851038e-12)

In [57]:
#77003 - EDO
race_count_hisp = zip_003 [zip_003 ["Race"]== "Count Hisp/Latino"]["Race Count"]
race_count_black = zip_003[zip_003["Race"]== "Counb Black/AfAm"]["Race Count"]
race_count_white = zip_003[zip_003["Race"]== "Count White"]["Race Count"]
race_count_asian = zip_003[zip_003["Race"]== "Count Asian"]["Race Count"]
race_count_AmInd = zip_003[zip_003["Race"]== "Count AmIndian"]["Race Count"]
race_count_other = zip_003[zip_003["Race"]== "Count Other Race"]["Race Count"]

#77007 - Paper Coffee Co.
race_count_hisp_zip007 = zip_007 [zip_007 ["Race"]== "Count Hisp/Latino"]["Race Count"]
race_count_black_zip007 = zip_007[zip_007["Race"]== "Counb Black/AfAm"]["Race Count"]
race_count_white_zip007 = zip_007[zip_007["Race"]== "Count White"]["Race Count"]
race_count_asian_zip007 = zip_007[zip_007["Race"]== "Count Asian"]["Race Count"]
race_count_AmInd_zip007 = zip_007[zip_007["Race"]== "Count AmIndian"]["Race Count"]
race_count_other_zip007 = zip_007[zip_007["Race"]== "Count Other Race"]["Race Count"]

#77008
race_count_hisp_zip008 = zip_008 [zip_008 ["Race"]== "Count Hisp/Latino"]["Race Count"]
race_count_black_zip008 = zip_008[zip_008["Race"]== "Counb Black/AfAm"]["Race Count"]
race_count_white_zip008 = zip_008[zip_008["Race"]== "Count White"]["Race Count"]
race_count_asian_zip008 = zip_008[zip_008["Race"]== "Count Asian"]["Race Count"]
race_count_AmInd_zip008 = zip_008[zip_008["Race"]== "Count AmIndian"]["Race Count"]
race_count_other_zip008 = zip_008[zip_008["Race"]== "Count Other Race"]["Race Count"]

#77026
race_count_hisp_zip026 = zip_026 [zip_026 ["Race"]== "Count Hisp/Latino"]["Race Count"]
race_count_black_zip026 = zip_026[zip_026["Race"]== "Counb Black/AfAm"]["Race Count"]
race_count_white_zip026 = zip_026[zip_026["Race"]== "Count White"]["Race Count"]
race_count_asian_zip026 = zip_026[zip_026["Race"]== "Count Asian"]["Race Count"]
race_count_AmInd_zip026 = zip_026[zip_026["Race"]== "Count AmIndian"]["Race Count"]
race_count_other_zip026 = zip_026[zip_026["Race"]== "Count Other Race"]["Race Count"]

In [58]:
ANOVA_race_count = stats.f_oneway(race_count_hisp, race_count_black, race_count_white, race_count_asian, 
                                  race_count_AmInd, race_count_other)

ANOVA_race_count_007 = stats.f_oneway(race_count_hisp_zip007, race_count_black_zip007, race_count_white_zip007, 
                                      race_count_asian_zip007, race_count_AmInd_zip007, race_count_other_zip007)

ANOVA_race_count_008 = stats.f_oneway(race_count_hisp_zip008, race_count_black_zip008, race_count_white_zip008, 
                                      race_count_asian_zip008, race_count_AmInd_zip008, race_count_other_zip008)

ANOVA_race_count_026 = stats.f_oneway(race_count_hisp_zip026, race_count_black_zip026, race_count_white_zip026, 
                                      race_count_asian_zip026, race_count_AmInd_zip026, race_count_other_zip026)

#this may not be the beat way of interpreting #there is a statistically significant difference between and within racial groups and zip codes from 2012 - 2017
#racial demographic matters when it comes to certain houston inner loop in zip codes 
print(ANOVA_race_count)
print(ANOVA_race_count_007)
print(ANOVA_race_count_008)
print(ANOVA_race_count_026)

F_onewayResult(statistic=847.5576251174277, pvalue=2.4659496131557764e-31)
F_onewayResult(statistic=615.053521180335, pvalue=2.896820800930107e-29)
F_onewayResult(statistic=662.4974619257885, pvalue=9.611320166924105e-30)
F_onewayResult(statistic=1188.126729265739, pvalue=1.6067634133914838e-33)


In [66]:
px.scatter(census_df, x="Household Income", y="Median Age", animation_frame="Year", animation_group="Zipcode",
           size="Population", color="Zipcode", hover_name="Zipcode",
           log_x=True, size_max=55, range_x=[20000,300000], range_y=[21,50])

In [52]:
#get years for our data
years = ["2012", "2013", "2014", "2015", "2016", "2017"]

# make list of zips
zipcodes = []
for zipcode in census_df["Zipcode"]:
    if zipcode not in zipcodes:
        zipcodes.append(zipcode)
# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}


# fill in most of layout
fig_dict["layout"]["xaxis"] = {"range": [10000, 300000], "title": "Household Income"}
fig_dict["layout"]["yaxis"] = {"title": "Median Age", "type": "log"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["sliders"] = {
    "args": [
        "transition", {
            "duration": 400,
            "easing": "cubic-in-out"
        }
    ],
    "initialValue": "2012",
    "plotlycommand": "animate",
    "values": years,
    "visible": True
}
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

#make data
year = 2012
for zipcode in zipcodes:
    dataset_by_year = census_df[census_df["Year"] == year]
    dataset_by_year_and_zip = dataset_by_year[
        dataset_by_year["Zipcode"] == zipcode]

    data_dict = {
        "x": list(dataset_by_year_and_zip["Household Income"]),
        "y": list(dataset_by_year_and_zip["Median Age"]),
        "mode": "markers",
        "text": list(dataset_by_year_and_zip["Zipcode"]),
        "marker": {
            "sizemode": "area",
            "sizeref": 100,
            "size": list(dataset_by_year_and_zip["Population"])
        },
        "name": zipcode
    }
    fig_dict["data"].append(data_dict)
        
    
# make frames 
for year in years:
    frame = {"data": [], "name": str(year)}
    for zipcode in zipcodes:
        dataset_by_year = census_df[census_df["Year"] == int(year)]
        dataset_by_year_and_zip = dataset_by_year[
            dataset_by_year["Zipcode"] == zipcode]

        data_dict = {
            "x": list(dataset_by_year_and_zip["Household Income"]),
            "y": list(dataset_by_year_and_zip["Median Age"]),
            "mode": "markers",
            "text": list(dataset_by_year_and_zip["Zipcode"]),
            "marker": {
                "sizemode": "area",
                "sizeref": 100,
                "size": list(dataset_by_year_and_zip["Population"])
            },
            "name": zipcode
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [year],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": year,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()


In [49]:
fig_dict

{'data': [{'x': [57165.0],
   'y': [31.8],
   'mode': 'markers',
   'text': [77002],
   'marker': {'sizemode': 'area', 'sizeref': 200000, 'size': [11473.0]},
   'name': 77002},
  {'x': [37664.0],
   'y': [32.9],
   'mode': 'markers',
   'text': [77003],
   'marker': {'sizemode': 'area', 'sizeref': 200000, 'size': [10215.0]},
   'name': 77003},
  {'x': [39804.0],
   'y': [31.6],
   'mode': 'markers',
   'text': [77004],
   'marker': {'sizemode': 'area', 'sizeref': 200000, 'size': [29853.0]},
   'name': 77004},
  {'x': [153041.0],
   'y': [38.5],
   'mode': 'markers',
   'text': [77005],
   'marker': {'sizemode': 'area', 'sizeref': 200000, 'size': [25740.0]},
   'name': 77005},
  {'x': [63360.0],
   'y': [36.9],
   'mode': 'markers',
   'text': [77006],
   'marker': {'sizemode': 'area', 'sizeref': 200000, 'size': [19856.0]},
   'name': 77006},
  {'x': [90860.0],
   'y': [34.6],
   'mode': 'markers',
   'text': [77007],
   'marker': {'sizemode': 'area', 'sizeref': 200000, 'size': [29545.0

In [31]:
url = "https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv"
dataset = pd.read_csv(url)

dataset.head()
# years = ["1952", "1962", "1967", "1972", "1977", "1982", "1987", "1992", "1997", "2002",
#          "2007"]

# # make list of continents
# continents = []
# for continent in dataset["continent"]:
#     if continent not in continents:
#         continents.append(continent)
# # make figure
# fig_dict = {
#     "data": [],
#     "layout": {},
#     "frames": []
# }

# # fill in most of layout
# fig_dict["layout"]["xaxis"] = {"range": [30, 85], "title": "Life Expectancy"}
# fig_dict["layout"]["yaxis"] = {"title": "GDP per Capita", "type": "log"}
# fig_dict["layout"]["hovermode"] = "closest"
# fig_dict["layout"]["sliders"] = {
#     "args": [
#         "transition", {
#             "duration": 400,
#             "easing": "cubic-in-out"
#         }
#     ],
#     "initialValue": "1952",
#     "plotlycommand": "animate",
#     "values": years,
#     "visible": True
# }
# fig_dict["layout"]["updatemenus"] = [
#     {
#         "buttons": [
#             {
#                 "args": [None, {"frame": {"duration": 500, "redraw": False},
#                                 "fromcurrent": True, "transition": {"duration": 300,
#                                                                     "easing": "quadratic-in-out"}}],
#                 "label": "Play",
#                 "method": "animate"
#             },
#             {
#                 "args": [[None], {"frame": {"duration": 0, "redraw": False},
#                                   "mode": "immediate",
#                                   "transition": {"duration": 0}}],
#                 "label": "Pause",
#                 "method": "animate"
#             }
#         ],
#         "direction": "left",
#         "pad": {"r": 10, "t": 87},
#         "showactive": False,
#         "type": "buttons",
#         "x": 0.1,
#         "xanchor": "right",
#         "y": 0,
#         "yanchor": "top"
#     }
# ]

# sliders_dict = {
#     "active": 0,
#     "yanchor": "top",
#     "xanchor": "left",
#     "currentvalue": {
#         "font": {"size": 20},
#         "prefix": "Year:",
#         "visible": True,
#         "xanchor": "right"
#     },
#     "transition": {"duration": 300, "easing": "cubic-in-out"},
#     "pad": {"b": 10, "t": 50},
#     "len": 0.9,
#     "x": 0.1,
#     "y": 0,
#     "steps": []
# }

# # make data
# year = 1952
# for continent in continents:
#     dataset_by_year = dataset[dataset["year"] == year]
#     dataset_by_year_and_cont = dataset_by_year[
#         dataset_by_year["continent"] == continent]

#     data_dict = {
#         "x": list(dataset_by_year_and_cont["lifeExp"]),
#         "y": list(dataset_by_year_and_cont["gdpPercap"]),
#         "mode": "markers",
#         "text": list(dataset_by_year_and_cont["country"]),
#         "marker": {
#             "sizemode": "area",
#             "sizeref": 200000,
#             "size": list(dataset_by_year_and_cont["pop"])
#         },
#         "name": continent
#     }
#     fig_dict["data"].append(data_dict)

# # make frames
# for year in years:
#     frame = {"data": [], "name": str(year)}
#     for continent in continents:
#         dataset_by_year = dataset[dataset["year"] == int(year)]
#         dataset_by_year_and_cont = dataset_by_year[
#             dataset_by_year["continent"] == continent]

#         data_dict = {
#             "x": list(dataset_by_year_and_cont["lifeExp"]),
#             "y": list(dataset_by_year_and_cont["gdpPercap"]),
#             "mode": "markers",
#             "text": list(dataset_by_year_and_cont["country"]),
#             "marker": {
#                 "sizemode": "area",
#                 "sizeref": 200000,
#                 "size": list(dataset_by_year_and_cont["pop"])
#             },
#             "name": continent
#         }
#         frame["data"].append(data_dict)

#     fig_dict["frames"].append(frame)
#     slider_step = {"args": [
#         [year],
#         {"frame": {"duration": 300, "redraw": False},
#          "mode": "immediate",
#          "transition": {"duration": 300}}
#     ],
#         "label": year,
#         "method": "animate"}
#     sliders_dict["steps"].append(slider_step)


# fig_dict["layout"]["sliders"] = [sliders_dict]

# fig = go.Figure(fig_dict)

# fig.show()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106
